## StarTracker TimeStamp check - 28-Nov-22

Craig Lage - basically copying code from Merlin Fisher-Levine

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver
# This requires the tickets/DM-31175 of summit_utils

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

In [ ]:
cameras = [['Wide', 101], ['Narrow', 102]]
dates = [16, 22, 23, 24]
for [camera, num] in cameras:
    for date in dates:
        if camera == 'Narrow' and date in [15, 16, 22]:
            # No narrow camera data in early dates
            continue
        
        #outfile = open(f"/scratch/cslage/starTracker/StarTracker_Summary_{camera}_202211{date}.txt", "w")
        #outfile.write("SeqNum\tRA(header)\tDec(header)\tRA(Solve)\tDec(Solve)\tAz(hdr-fix)\tEl(header)\tAz(solve)\tEl(solve)\tDeltaAz\tDeltaEl\tRot\tRMS(asec)\n")
        path = f"/project/GenericCamera/{num}/2022/11/{date}/"
        for seqNum in range(200):
            try:
                filename = path + f"GC{num}_O_202211{date}_{seqNum:06}.fits"
                hdulist = pf.open(filename)
                header = hdulist[0].header
                raStart = header["RASTART"]
                decStart = header["DECSTART"]
                azStart = header["AZSTART"]

                elStart = header["ELSTART"]
                rotpa = header["ROTPA"]
                time = Time(header["DATE-OBS"], scale='tai')
                start = time.utc - TimeDelta(60.0, format='sec')
                end = time.utc + TimeDelta(60.0, format='sec')
                df = await client.select_time_series('lsst.sal.GenericCamera.logevent_startIntegration', \
                                                    ['*'],  start, end)
                imageNumber = df['imageNumber'].loc[(df['salIndex'] == num) & (df['imageNumber'] == seqNum)]
                timeIndex = imageNumber.index
                deltaT = (time-Time(timeIndex))
                print(date, num, seqNum, time, len(timeIndex), timeIndex[0], deltaT.sec[0])
            except:
                continue

            
    

In [ ]:
time

In [ ]:
Time(timeIndex)

In [ ]:
delta = (time-Time(timeIndex))

In [ ]:
delta

In [ ]:
delta.sec